In [4]:
# Установка необходимых библиотек
!pip install biopython
!pip install bcbio-gff

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 2.2 MB/s eta 0:00:00


In [6]:
main_fasta_filename = "/content/ncbi_dataset/data/GCA_017916305.1/GCA_017916305.1_LU_Lenr_1.0_genomic.fna"
dna_seq_filename = "dnaSeq.txt"
annotation_filename = "/content/ncbi_dataset/data/GCA_017916305.1/genomic.gff"

In [7]:
from Bio import SeqIO
res = ""
count = 0
for record in SeqIO.parse(main_fasta_filename, "fasta"):
    res += str(record.seq)
    print(record.id)
    print(repr(record.seq))
    print(len(record))
    print(len(record.seq))
print()
print(f"{len(res) / 1024 / 1024:.2f} MB")
with open(dna_seq_filename, "w") as f:
    f.write(res.upper())

CM030576.1
Seq('TCGCCGCATCGGTGCAGCAGATGGCGTCGTACGCGCTGTCGTCATGGGTGGCCA...gga')
263205
263205
CM030577.1
Seq('gtgagctgctggcgctcctgctgGCCGCATGCGTTGAACAGCACTCCTCCACAT...AAT')
458352
458352
CM030578.1
Seq('GATCTCGTGCGCTCGTGGGCCCGAAGCGCTAGCGACAATGgcacctctgtgtgt...gca')
413733
413733
CM030579.1
Seq('CTTAATTCTGACTTGACCTGACTCGACTTGACTTGACTTTAAttcttaatttta...TCT')
502684
502684
CM030580.1
Seq('AAAAAAATTCCAAAAAAAATTCCAAAAAAAAGCGACTAACAAACACAACTGATG...ACT')
491206
491206
CM030581.1
Seq('TTgacttgactttaattttaactttaactttaattttaattttaattttaattt...GGC')
524723
524723
CM030582.1
Seq('TTTCGTTTCTGCATGCATCTTTCTTATCTCATGTACTCTCTGGTCCAGCCAGCG...GCG')
676625
676625
CM030583.1
Seq('TTGAGGGGATtatggaggcacacaacgcggccgctaaggcagccgcacgccgcc...TTT')
509413
509413
CM030584.1
Seq('ACCCCCATCgcaccgtttctctctctctctgaaacgtcgacgagggactcggca...GAG')
646429
646429
CM030585.1
Seq('CACGCGGATTCagcagtggtgctgctgaacgGTGAAGCCGACagggcacacgca...gta')
638765
638765
CM030586.1
Seq('CGACGAGCGTCAACGCGGGGCGACTCTGTCGAcgcgatctctgctgacttGGCT

In [ ]:
#with open("otterDnaSeq.txt", "w") as f:
#    f.write("A")
import subprocess
result = subprocess.run(["zhunt3.exe", "12", "8", "12", dna_seq_filename], stdout=subprocess.PIPE)
print(result.stdout)

b'dinucleotides 12\r\nmin/max 8 12\r\nmin/max 8 12\r\noperating on dnaSeq.txt\r\ncalculating zscore\r\nopening dnaSeq.txt\r\ninputting sequence\r\nopening dnaSeq.txt.Z-SCORE\r\n\r\n run time=1718553843 sec\r\nuse min/max 8 12\r\nanalyzing_zscore\r\nopening dnaSeq.txt.Z-SCORE\r\nopening dnaSeq.txt\r\ninputting sequence\r\n'


In [26]:
# prepping z-score file for parsing
z_score_filename = "zhunt_results_filtered_1k.csv"
with open(z_score_filename, "r") as f:
    f.readline()
    datastr = f.read()
print(f"data read: {len(datastr) / 1024 / 1024:.2f} MB")

data read: 57.87 MB


In [ ]:
# writing z-score file without heading
# print(f"Writing tsv file: {len(datastr) / 1024 / 1024:.2f} MB")
# with open(f"{dna_seq_filename}.Z-SCORE.tsv", "w") as f:
#     f.write(datastr)

Writing tsv file: 1.37 MB


In [31]:
print(f"Reading csv file")
z_dna_datas = list()
with open(z_score_filename, "r") as f:
    for line in f:
        d = line.split()
        if len(d) < 6:
            continue
        data = list(map(int, d[:2]))
        z_dna_datas.append(data)
print(len(z_dna_datas))

# group adjacent regions
z_dna_grouped = list()
z_dna_grouped.append(z_dna_datas[0])
last_left_idx = z_dna_grouped[0][1]
for i in range(1, len(z_dna_datas)):
    if z_dna_datas[i][0] > z_dna_datas[i - 1][1]:
        z_dna_grouped.append(z_dna_datas[i])
        last_left_idx = z_dna_datas[i][1]
    else:
        if (z_dna_datas[i][0] > last_left_idx):
            last_left_idx = z_dna_datas[i][1]
            z_dna_grouped.append(z_dna_datas[i])
print(len(z_dna_grouped))
# use grouped z-dnas
z_dna_datas = z_dna_grouped

Reading csv file
647793
59109


In [11]:
# count quadruplexes
import re

pattern_G = "(?:G{3,}[ATGC]{1,7}){3,}G{3,}"
pattern_C = "(?:C{3,}[ATGC]{1,7}){3,}C{3,}"
pattern_checker = "GGG"

with open(dna_seq_filename, 'r') as f:
    dna_seq = f.read()
    PQS=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern_G,dna_seq,re.IGNORECASE)]
    print("+ strand :", len(PQS))
    quad_on_plus = PQS
    PQS=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern_C,dna_seq,re.IGNORECASE)]
    print("- strand :", len(PQS))
    quad_on_minus = PQS
    PQS=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern_checker,dna_seq,re.IGNORECASE)]
    print("checker :", len(PQS))

+ strand : 4417
- strand : 4478
checker : 405235


In [22]:
genes_data = []
exons_data = []
chromosome_sizes = []
global_offset = 0
stored_prev_range_offset = 0
with open(annotation_filename, 'r') as f:
    for line in f.readlines():
        if line.startswith("#"):
            continue

        # 2 - type 3 - begin 4 - end 6 - strand
        data = line.strip("\n").split("\t")
        if data[2] == 'region':
            chromosome_sizes.append(int(data[4]))
            global_offset += stored_prev_range_offset
            stored_prev_range_offset = int(data[4])
            continue
        if data[2] == 'exon':
            exons_data.append([data[6], global_offset + int(data[3]), global_offset + int(data[4])])
            continue
        if data[2] == 'gene':
            genes_data.append([data[6], global_offset + int(data[3]), global_offset + int(data[4])])
            continue

gene_downstream_regions = [[gene_data[2], gene_data[2] + 200] if (gene_data[0] == "+") else [gene_data[1] - 200, gene_data[1]] for gene_data in genes_data]
gene_promoter_regions = [[gene_data[1] - 1000, gene_data[1]] if (gene_data[0] == "+") else [gene_data[2], gene_data[2] + 1000] for gene_data in genes_data]
in_gene_regions = [[gene_data[1], gene_data[2]] for gene_data in genes_data]
exon_regions = [[exon_data[1], exon_data[2]] for exon_data in exons_data]

In [32]:
def bin_search_has_region(segment, regions):
    l = 0
    r = len(regions)
    middle_of_segment = (segment[0] + segment[1]) / 2
    while(True):
        if (l >= r):
            return False
        middle = (l + r) // 2
        if (regions[middle][0] <= middle_of_segment <= regions[middle][1]):
            return True
        if (middle_of_segment < regions[middle][0]):
            r = middle
        else:
            l = middle + 1

in_dna_count = 0
in_promoter_count = 0
in_downstream_count = 0
in_exon_count = 0
in_intergenic_count = 0

for z_dna in z_dna_datas:
    inSomewhere = False
    if bin_search_has_region(z_dna, gene_promoter_regions):
        in_promoter_count += 1
        inSomewhere = True
    if bin_search_has_region(z_dna, gene_downstream_regions):
        in_downstream_count += 1
        inSomewhere = True
    if bin_search_has_region(z_dna, exon_regions):
        in_exon_count += 1
        inSomewhere = True
    if bin_search_has_region(z_dna, in_gene_regions):
        in_dna_count += 1
        inSomewhere = True
    if not inSomewhere:
        in_intergenic_count += 1

z_dna_count = len(z_dna_grouped)
print(f"In exons:", in_exon_count, "доля:", in_exon_count / z_dna_count)
print(f"In promoters:", in_promoter_count, "доля:", in_promoter_count / z_dna_count)
print(f"In downstream:", in_downstream_count, "доля:", in_downstream_count / z_dna_count)
print(f"In intergenic space:", in_intergenic_count, "доля:", in_intergenic_count / z_dna_count)
print(f"In dna:", in_dna_count, "доля:", in_dna_count / z_dna_count)



In exons: 19351 доля: 0.3273782334331489
In promoters: 17722 доля: 0.29981897849735234
In downstream: 3150 доля: 0.05329137694767294
In intergenic space: 20252 доля: 0.34262125902992774
In dna: 20413 доля: 0.3453450405183644


In [18]:
in_dna_count = 0
in_promoter_count = 0
in_downstream_count = 0
in_exon_count = 0
in_intergenic_count = 0

for quad in quad_on_plus:
    inSomewhere = False
    if bin_search_has_region(quad, gene_promoter_regions):
        in_promoter_count += 1
        inSomewhere = True
    if bin_search_has_region(quad, gene_downstream_regions):
        in_downstream_count += 1
        inSomewhere = True
    if bin_search_has_region(quad, exon_regions):
        in_exon_count += 1
        inSomewhere = True
    if bin_search_has_region(quad, in_gene_regions):
        in_dna_count += 1
        inSomewhere = True
    if not inSomewhere:
        in_intergenic_count += 1

quad_count = len(quad_on_plus)
print(f"In exons:", in_exon_count, "доля:", in_exon_count / quad_count)
print(f"In promoters:", in_promoter_count, "доля:", in_promoter_count / quad_count)
print(f"In downstream:", in_downstream_count, "доля:", in_downstream_count / quad_count)
print(f"In intergenic space:", in_intergenic_count, "доля:", in_intergenic_count / quad_count)
print(f"In dna:", in_dna_count, "доля:", in_dna_count / quad_count)

In exons: 196 доля: 0.044374009508716325
In promoters: 1812 доля: 0.41023318994792846
In downstream: 347 доля: 0.0785601086710437
In intergenic space: 2119 доля: 0.4797373783110709
In dna: 269 доля: 0.060901064070636175


In [24]:
zdnabert_res_filename = "/content/text_predictions.txt"

z_dna_datas = []
global_offset = 0
chrom_idx = 0
with open(zdnabert_res_filename, 'r') as f:
    f.readline()
    for line in f:
        if line.startswith("CM"):
            continue
        if line.strip().startswith("start"):
            global_offset += chromosome_sizes[chrom_idx]
            chrom_idx += 1
            continue

        data = line.strip().split()
        data = list(map(int, line.split()[:2]))
        data[0] += global_offset
        data[1] += global_offset
        z_dna_datas.append(data)

print(len(z_dna_datas))

# group adjacent regions
z_dna_grouped = list()
z_dna_grouped.append(z_dna_datas[0])
last_left_idx = z_dna_grouped[0][1]
for i in range(1, len(z_dna_datas)):
    if z_dna_datas[i][0] > z_dna_datas[i - 1][1]:
        z_dna_grouped.append(z_dna_datas[i])
        last_left_idx = z_dna_datas[i][1]
    else:
        if (z_dna_datas[i][0] > last_left_idx):
            last_left_idx = z_dna_datas[i][1]
            z_dna_grouped.append(z_dna_datas[i])
print(len(z_dna_grouped))
# use grouped z-dnas
z_dna_datas = z_dna_grouped

102670
102659


In [25]:
in_dna_count = 0
in_promoter_count = 0
in_downstream_count = 0
in_exon_count = 0
in_intergenic_count = 0

for z_dna in z_dna_datas:
    inSomewhere = False
    if bin_search_has_region(z_dna, gene_promoter_regions):
        in_promoter_count += 1
        inSomewhere = True
    if bin_search_has_region(z_dna, gene_downstream_regions):
        in_downstream_count += 1
        inSomewhere = True
    if bin_search_has_region(z_dna, exon_regions):
        in_exon_count += 1
        inSomewhere = True
    if bin_search_has_region(z_dna, in_gene_regions):
        in_dna_count += 1
        inSomewhere = True
    if not inSomewhere:
        in_intergenic_count += 1

z_dna_count = len(z_dna_grouped)
print(f"In exons:", in_exon_count, "доля:", in_exon_count / z_dna_count)
print(f"In promoters:", in_promoter_count, "доля:", in_promoter_count / z_dna_count)
print(f"In downstream:", in_downstream_count, "доля:", in_downstream_count / z_dna_count)
print(f"In intergenic space:", in_intergenic_count, "доля:", in_intergenic_count / z_dna_count)
print(f"In dna:", in_dna_count, "доля:", in_dna_count / z_dna_count)

In exons: 44259 доля: 0.4311263503443439
In promoters: 23303 доля: 0.22699422359461907
In downstream: 4755 доля: 0.04631839390603844
In intergenic space: 32868 доля: 0.32016676570003605
In dna: 45685 доля: 0.4450169980225796
